# Try Tiny BNN with MNIST

## 1. Dataset Loading

In [1]:
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader, random_split

In [2]:
full_data_train = torchvision.datasets.MNIST('./data/', 
                                        train=True, download=True,
                                        transform=torchvision.transforms.Compose
                                        ([
                                            torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize((0.5,), (0.5,))
                                        ]))

# Split the dataset into training and validation subsets
train_size = int(0.8 * len(full_data_train))
val_size = len(full_data_train) - train_size
data_train, data_valid = random_split(full_data_train, [train_size, val_size])

data_test = torchvision.datasets.MNIST('./data/', 
                                       train=False, download=True,
                                       transform=torchvision.transforms.Compose
                                       ([
                                            torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize((0.5,), (0.5,))
                                       ]))

## 2. Define MLP structure

In [3]:
import math
import torch
import torch.nn as nn
from torch.nn import Module
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm, trange

In [4]:
import brevitas.nn as qnn
from brevitas.nn import QuantLinear, QuantReLU, QuantConv2d
from brevitas.quant.binary import SignedBinaryActPerTensorConst
from brevitas.quant.binary import SignedBinaryWeightPerTensorConst
from brevitas.inject.enum import QuantType

In [5]:
input_size = 1*28*28      
hidden1 = 512      
hidden2 = 512
hidden3 = 512
num_classes = 10  

In [6]:
class BiKA_MNIST(Module):
    def __init__(self):
        super(BiKA_MNIST, self).__init__()
        
        self.input = qnn.QuantIdentity(quant_type='int', scaling_impl_type='const', bit_width=8, min_val=-128.0, max_val=127.0, return_quant_tensor=True)
        
        self.fc0   = qnn.QuantLinear(input_size, hidden1, weight_bit_width=8, weight_quant_type=QuantType.INT, bias=False)
        self.bn0   = nn.BatchNorm1d(hidden1)
        self.relu0 = qnn.QuantReLU(bit_width=8, return_quant_tensor=True)
        
        self.fc1   = qnn.QuantLinear(hidden1, hidden2, weight_bit_width=8, weight_quant_type=QuantType.INT, bias=False)
        self.bn1   = nn.BatchNorm1d(hidden2)
        self.relu1 = qnn.QuantReLU(bit_width=8, return_quant_tensor=True)
        
        self.fc2   = qnn.QuantLinear(hidden2, hidden3, weight_bit_width=8, weight_quant_type=QuantType.INT, bias=False)
        self.bn2   = nn.BatchNorm1d(hidden3)
        self.relu2 = qnn.QuantReLU(bit_width=8, return_quant_tensor=True)
        
        self.out   = qnn.QuantLinear(hidden3, num_classes, weight_bit_width=8, weight_quant_type=QuantType.INT, bias=False)

    def forward(self, x):
        
        out = x.reshape(x.shape[0], -1)
        out = self.input(out)
        out = self.relu0(self.bn0(self.fc0(out)))
        out = self.relu1(self.bn1(self.fc1(out)))
        out = self.relu2(self.bn2(self.fc2(out)))
        out = self.out(out)
        
        return out

## 3. Define Training Function

In [7]:
num_of_gpus = torch.cuda.device_count()
print(num_of_gpus)

# Check for GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Setting seeds for reproducibility
torch.manual_seed(0)

2
Using device: cuda:0


In [8]:
def display_loss_plot(losses, title="Training loss", xlabel="Iterations", ylabel="Loss"):
    x_axis = [i for i in range(len(losses))]
    plt.plot(x_axis,losses)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

In [9]:
def train_and_validate(model, train_loader, val_loader, criterion, learning_rate):
    model.train()
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate[0])
    
    for epoch in range(100):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
        # Adjust learning rate at epoch 100
        if epoch+1 == 50:
            for param_group in optimizer.param_groups:
                param_group['lr'] = learning_rate[1]
                print(f"Learning rate changed to {param_group['lr']} at epoch {epoch+1}")
        
        # Adjust learning rate at epoch 150
        if epoch+1 == 75:
            for param_group in optimizer.param_groups:
                param_group['lr'] = learning_rate[2]
                print(f"Learning rate changed to {param_group['lr']} at epoch {epoch+1}")

        # Validation phase
        model.eval()
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_acc = accuracy_score(all_labels, all_preds)
        print(f"Epoch [{epoch+1}/{100}], "
              f"Train Loss: {running_loss/len(train_loader):.4f}, "
              f"Val Accuracy: {val_acc*100:.2f}%")
        
    return val_acc

## 4. Define Evaluation Function

In [10]:
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    test_acc = accuracy_score(all_labels, all_preds)
    print(f"Test Accuracy: {test_acc * 100:.2f}%")
    return test_acc

## 5. Train BNN for MNIST

In [11]:
batch_sizes = [64, 128, 256]
learning_rates = [[0.0100, 0.0010, 0.0010],
                  [0.0010, 0.0010, 0.0010],
                  [0.0010, 0.0010, 0.0001],
                  [0.0010, 0.0005, 0.0001]
                 ]

In [12]:
best_acc = 0.0
best_params = None

for batch_size, learning_rate in product(batch_sizes, learning_rates):
    print(f"Training with batch_size={batch_size}, learning_rate={learning_rate}")

    # Data loaders
    train_loader = DataLoader(data_train, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(data_valid, batch_size=batch_size, shuffle=False)

    # Initialize the model, loss, and optimizer
    model = BiKA_MNIST().to(device)
    criterion = nn.CrossEntropyLoss()

    # Train and validate
    val_acc = train_and_validate(model, train_loader, val_loader, criterion, learning_rate)

    # Update best parameters
    if val_acc > best_acc:
        best_acc = val_acc
        best_params = (batch_size, learning_rate)

print(f"Best Accuracy: {best_acc*100:.2f}%")
print(f"Best Parameters: Batch Size={best_params[0]}, Learning Rate={best_params[1]}")

Training with batch_size=64, learning_rate=[0.01, 0.001, 0.001]


/home/yuhao/anaconda3/envs/brevitas_lyh/lib/python3.9/site-packages/torch/_tensor.py:1362: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/c10/core/TensorImpl.h:1900.)
  return super().rename(names)


/home/yuhao/anaconda3/envs/brevitas_lyh/lib/python3.9/site-packages/brevitas/nn/quant_linear.py:69: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/torch/csrc/utils/python_arg_parser.cpp:368.)
  output_tensor = linear(x, quant_weight, quant_bias)


Epoch [1/100], Train Loss: 0.2670, Val Accuracy: 95.37%


/home/yuhao/anaconda3/envs/brevitas_lyh/lib/python3.9/site-packages/brevitas/nn/quant_linear.py:69: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/torch/csrc/utils/python_arg_parser.cpp:368.)
  output_tensor = linear(x, quant_weight, quant_bias)


...

Epoch [98/100], Train Loss: 0.0000, Val Accuracy: 98.18%


/home/yuhao/anaconda3/envs/brevitas_lyh/lib/python3.9/site-packages/brevitas/nn/quant_linear.py:69: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/torch/csrc/utils/python_arg_parser.cpp:368.)
  output_tensor = linear(x, quant_weight, quant_bias)


Epoch [99/100], Train Loss: 0.0000, Val Accuracy: 98.18%


/home/yuhao/anaconda3/envs/brevitas_lyh/lib/python3.9/site-packages/brevitas/nn/quant_linear.py:69: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/torch/csrc/utils/python_arg_parser.cpp:368.)
  output_tensor = linear(x, quant_weight, quant_bias)


Epoch [100/100], Train Loss: 0.0000, Val Accuracy: 98.18%
Best Accuracy: 98.18%
Best Parameters: Batch Size=256, Learning Rate=[0.001, 0.0005, 0.0001]


## 6. Evaluate BNN for MNIST

In [13]:
train_loader = DataLoader(data_train, batch_size=best_params[0], shuffle=True)
val_loader = DataLoader(data_valid, batch_size=best_params[0], shuffle=False)
test_loader = DataLoader(data_test, batch_size=best_params[0], shuffle=False)

model = BiKA_MNIST().to(device)
criterion = nn.CrossEntropyLoss()

train_and_validate(model, train_loader, val_loader, criterion, best_params[1])

/home/yuhao/anaconda3/envs/brevitas_lyh/lib/python3.9/site-packages/brevitas/nn/quant_linear.py:69: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/torch/csrc/utils/python_arg_parser.cpp:368.)
  output_tensor = linear(x, quant_weight, quant_bias)


Epoch [1/100], Train Loss: 0.2003, Val Accuracy: 96.08%


/home/yuhao/anaconda3/envs/brevitas_lyh/lib/python3.9/site-packages/brevitas/nn/quant_linear.py:69: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/torch/csrc/utils/python_arg_parser.cpp:368.)
  output_tensor = linear(x, quant_weight, quant_bias)


Epoch [2/100], Train Loss: 0.3066, Val Accuracy: 95.93%


/home/yuhao/anaconda3/envs/brevitas_lyh/lib/python3.9/site-packages/brevitas/nn/quant_linear.py:69: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/torch/csrc/utils/python_arg_parser.cpp:368.)
  output_tensor = linear(x, quant_weight, quant_bias)


...

/home/yuhao/anaconda3/envs/brevitas_lyh/lib/python3.9/site-packages/brevitas/nn/quant_linear.py:69: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/torch/csrc/utils/python_arg_parser.cpp:368.)
  output_tensor = linear(x, quant_weight, quant_bias)


Epoch [99/100], Train Loss: 0.0000, Val Accuracy: 98.12%


/home/yuhao/anaconda3/envs/brevitas_lyh/lib/python3.9/site-packages/brevitas/nn/quant_linear.py:69: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/torch/csrc/utils/python_arg_parser.cpp:368.)
  output_tensor = linear(x, quant_weight, quant_bias)


Epoch [100/100], Train Loss: 0.0000, Val Accuracy: 98.16%


0.9815833333333334

In [14]:
print(f"Best Validation Accuracy: {best_acc*100:.2f}%")
print(f"Best Parameters: Batch Size={best_params[0]}, Learning Rate={best_params[1]}")

evaluate_model(model, test_loader)

Best Validation Accuracy: 98.18%
Best Parameters: Batch Size=256, Learning Rate=[0.001, 0.0005, 0.0001]


/home/yuhao/anaconda3/envs/brevitas_lyh/lib/python3.9/site-packages/brevitas/nn/quant_linear.py:69: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/torch/csrc/utils/python_arg_parser.cpp:368.)
  output_tensor = linear(x, quant_weight, quant_bias)


Test Accuracy: 98.35%


0.9835